# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [70]:
import os, ssl
# os.getcwd()
os.chdir('/Users/apple/PycharmProjects/disaster_response_pipeline/project')

In [6]:

# if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
# getattr(ssl, '_create_unverified_context', None)):
# ssl._create_default_https_context = ssl._create_unverified_context

In [28]:
# import libraries
import nltk
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from nltk.corpus import stopwords, wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import string
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import os

In [72]:
# load data from database
df = pd.read_sql_table('disaster_data', 'sqlite:///data/myDatabase.db')
X = df.message
Y = df.drop(['id', 'original', 'message', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [75]:
stopwords_ = stopwords.words('english')

def tokenize(text):
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Reduce words to their root form
    words = text.split()
    words = [w for w in words if w not in stopwords_]
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return lemmed


In [76]:
corpus = X.apply(lambda x: tokenize(x))

In [82]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [84]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3), tokenizer=tokenize)),
#         ('vect', CountVectorizer(tokenizer=tokenize)),
#         ('tfidf', TfidfTransformer(ngrams = 1, minDocFreq = 2, wordLengths = 3,
#                                    wordLengths_max = 15, idf = True, cores = 6)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42))),
        ])
#         ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# predict on test data
Y_pred = pipeline.predict(X_test)
Y_pred.shape()


In [ ]:
# display results
colnames = Y_test.columns
Y_pred = pd.DataFrame(Y_pred, columns=colnames)
for col in colnames:
    print(col)
    print(classification_report(Y_test[col], Y_pred[col]))


### 6. Improve your model
Use grid search to find better parameters. 

In [41]:
parameters = { 
    
    'clf__estimator__n_estimators': [200, 500],
    'clf__estimator__max_features': ['auto', 'sqrt', 'log2'],
    'clf__estimator__max_depth' : [4,5,6,7,8],
    'clf__estimator__criterion' :['gini', 'entropy']
}


cv = (GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=-1))

In [42]:
cv.fit(X_train, Y_train)

cv.best_estimator_    

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x135c0fe50>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=4,
                                                                        n_estimators=200,
                                                                        random_state=42)))])

In [47]:
pd.DataFrame(cv.cv_results_).sort_values('rank_test_score').head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator__criterion,param_clf__estimator__max_depth,param_clf__estimator__max_features,param_clf__estimator__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,104.572065,0.247434,16.161192,0.260192,gini,4,auto,200,"{'clf__estimator__criterion': 'gini', 'clf__es...",0.200356,0.196034,0.192777,0.196338,0.192014,0.195504,0.002971,1
28,97.197853,0.645405,16.252276,1.597667,gini,8,log2,200,"{'clf__estimator__criterion': 'gini', 'clf__es...",0.200356,0.196034,0.192777,0.196338,0.192014,0.195504,0.002971,1
58,1682.438475,1962.489648,23.175879,12.624546,entropy,8,log2,200,"{'clf__estimator__criterion': 'entropy', 'clf_...",0.200356,0.196034,0.192777,0.196338,0.192014,0.195504,0.002971,1
30,105.925848,1.190372,14.271268,0.189740,entropy,4,auto,200,"{'clf__estimator__criterion': 'entropy', 'clf_...",0.200356,0.196034,0.192777,0.196338,0.192014,0.195504,0.002971,1
31,243.030561,1.781384,31.037262,0.619556,entropy,4,auto,500,"{'clf__estimator__criterion': 'entropy', 'clf_...",0.200356,0.196034,0.192777,0.196338,0.192014,0.195504,0.002971,1
32,103.646830,0.839303,13.945809,0.433737,entropy,4,sqrt,200,"{'clf__estimator__criterion': 'entropy', 'clf_...",0.200356,0.196034,0.192777,0.196338,0.192014,0.195504,0.002971,1
33,246.747323,0.756822,30.992404,0.427396,entropy,4,sqrt,500,"{'clf__estimator__criterion': 'entropy', 'clf_...",0.200356,0.196034,0.192777,0.196338,0.192014,0.195504,0.002971,1
34,89.460172,1.032694,14.289293,0.487977,entropy,4,log2,200,"{'clf__estimator__criterion': 'entropy', 'clf_...",0.200356,0.196034,0.192777,0.196338,0.192014,0.195504,0.002971,1
23,216.290121,2.503552,31.536762,1.237986,gini,7,log2,500,"{'clf__estimator__criterion': 'gini', 'clf__es...",0.200356,0.196034,0.192777,0.196338,0.192014,0.195504,0.002971,1
35,205.384241,0.827407,31.491158,0.873502,entropy,4,log2,500,"{'clf__estimator__criterion': 'entropy', 'clf_...",0.200356,0.196034,0.192777,0.196338,0.192014,0.195504,0.002971,1


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [48]:
model = cv.best_estimator_
Y_pred = model.predict(X_test)


In [ ]:
Y_pred[0]

In [49]:

Y_pred = pd.DataFrame(Y_pred, columns=colnames)
for col in colnames:
    print(col)
    print(classification_report(Y_test[col], Y_pred[col]))


related
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1507
         1.0       0.76      1.00      0.87      5006
         2.0       0.00      0.00      0.00        41

    accuracy                           0.76      6554
   macro avg       0.25      0.33      0.29      6554
weighted avg       0.58      0.76      0.66      6554

request
              precision    recall  f1-score   support

         0.0       0.84      1.00      0.91      5475
         1.0       0.00      0.00      0.00      1079

    accuracy                           0.84      6554
   macro avg       0.42      0.50      0.46      6554
weighted avg       0.70      0.84      0.76      6554

offer


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6524
         1.0       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

aid_related
              precision    recall  f1-score   support

         0.0       0.59      1.00      0.74      3841
         1.0       0.92      0.00      0.01      2713

    accuracy                           0.59      6554
   macro avg       0.75      0.50      0.37      6554
weighted avg       0.72      0.59      0.44      6554

medical_help
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      6044
         1.0       0.00      0.00      0.00       510

    accuracy                           0.92      6554
   macro avg       0.46      0.50      0.48      6554
weighted avg       0.85      0.92      0.88      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [50]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.